# Ultimate MIDI Classifier Maker (ver. 1.0)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

WARNING: This complete implementation is a functioning model of the Artificial Intelligence. Please excercise great humility, care, and respect. https://www.nscai.gov/

***

#### Project Los Angeles

#### Tegridy Code 2024

***

# GPU check

In [ ]:
!nvidia-smi

# Setup environment

In [ ]:
!git clone --depth 1 https://github.com/asigalov61/tegridy-tools
!pip3 install torch torchvision torchaudio
!pip install einops
!pip install torch-summary

# Import modules

In [ ]:
# Load modules and make data dir

print('Loading modules...')

import os
import pickle
import random
import secrets
import tqdm
import math
import torch
import torch.optim as optim

from torch.utils.data import DataLoader, Dataset

import matplotlib.pyplot as plt

from torchsummary import summary
from sklearn import metrics

%cd /home/ubuntu/tegridy-tools/tegridy-tools/

import TMIDIX

%cd /home/ubuntu/tegridy-tools/tegridy-tools/X-Transformer

from x_transformer_1_23_2 import *

torch.set_float32_matmul_precision('high')
torch.backends.cuda.matmul.allow_tf32 = True # allow tf32 on matmul
torch.backends.cudnn.allow_tf32 = True # allow tf32 on cudnn

%cd /home/ubuntu/

if not os.path.exists('/home/ubuntu/INTS'):
    os.makedirs('/home/ubuntu/INTS')

import random

print('Done')

print('Torch version:', torch.__version__)

# Load training data

# Files List

In [ ]:
dataset_addr = "/home/ubuntu/INTS"

#==========================================================================

filez = list()
for (dirpath, dirnames, filenames) in os.walk(dataset_addr):
    filez += [os.path.join(dirpath, file) for file in filenames]
print('=' * 70)

random.shuffle(filez)

print('Loaded', len(filez), 'data files')
print('=' * 70)

# Load Training Data

In [ ]:
SEQ_LEN = 1026 # Models seq len (must be divisible by 4)
PAD_IDX = 940 # Models pad index

CHUNKS_LENGTH = SEQ_LEN+1

#==========================================================================

print('=' * 70)
print('Loading data files...')
print('Please wait...')
print('=' * 70)

train_data = []

chunk_size = 1020

for lfa in tqdm.tqdm(filez):

    train_d = pickle.load(open(lfa, 'rb'))
    random.shuffle(train_d)
    for ta in train_d:

        fnames_toks = ta[0]
        score = ta[1]

        for i in range(0, len(score)-chunk_size, chunk_size // 2):
            schunk = score[i:i+chunk_size]

            if len(schunk) == chunk_size:

                td = [937]

                td.extend(schunk)

                td.extend([938])

                td.extend([fnames_toks[0]+512, fnames_toks[1]+512])

                td.extend([939])

                train_data.append(td)

#==========================================================================

print('Done!')
print('=' * 70)
print('Total number of good chunks:', len(train_data))
print('All data is good:', len(max(train_data, key=len)) == len(min(train_data, key=len)))
print('=' * 70)
print('Final data randomization...')
random.shuffle(train_data)
print('Done!')
print('=' * 70)

In [ ]:
train_data[0][-15:], len(train_data[0])

# Setup model

In [ ]:
# Setup model

# constants

NUM_EPOCHS = 1

BATCH_SIZE = 88
GRADIENT_ACCUMULATE_EVERY = 1

LEARNING_RATE = 1e-4

TOTAL_STEPS = int((len(train_data) / (BATCH_SIZE * GRADIENT_ACCUMULATE_EVERY)) * NUM_EPOCHS)

VALIDATE_EVERY  = 100
SAVE_EVERY = 500
GENERATE_EVERY  = 250
GENERATE_LENGTH = 4
PRINT_STATS_EVERY = 20

# helpers

def cycle(loader):
    while True:
        for data in loader:
            yield data

# instantiate the model

model = TransformerWrapper(
    num_tokens = PAD_IDX+1,
    max_seq_len = SEQ_LEN,
    attn_layers = Decoder(dim = 1024, depth = 24, heads = 32, attn_flash = True)
    )

model = AutoregressiveWrapper(model, ignore_index = PAD_IDX, pad_value=PAD_IDX)

model = torch.nn.DataParallel(model)

model.cuda()

print('Done!')

summary(model)

# Dataloader

class MusicDataset(Dataset):
    def __init__(self, data, seq_len):
        super().__init__()
        self.data = data
        self.seq_len = seq_len

    def __getitem__(self, index):

        # consequtive sampling

        full_seq = torch.Tensor(self.data[index][:self.seq_len+1]).long()

        return full_seq.cuda()

    def __len__(self):
        return (len(self.data) // BATCH_SIZE) * BATCH_SIZE

# precision/optimizer/scaler

optim = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train

In [ ]:
# Train the model

train_losses = []
val_losses = []

train_accs = []
val_accs = []

nsteps = 0

for da in range(5):

        tdata = train_data

        random.shuffle(tdata)

        print('=' * 70)
        print('Epoch', da)
        print('=' * 70)

        train_dataset = MusicDataset(tdata, SEQ_LEN)
        val_dataset   = MusicDataset(tdata, SEQ_LEN)
        train_loader  = cycle(DataLoader(train_dataset, batch_size = BATCH_SIZE))
        val_loader    = cycle(DataLoader(val_dataset, batch_size = BATCH_SIZE))

        NUM_BATCHES = (len(tdata) // BATCH_SIZE // GRADIENT_ACCUMULATE_EVERY) * NUM_EPOCHS

        model.train()

        for i in tqdm.tqdm(range(NUM_BATCHES), mininterval=10., desc='Training'):

            optim.zero_grad(set_to_none=True)

            loss, acc = model(next(train_loader))
            loss.backward(torch.ones(loss.shape).cuda())

            if i % PRINT_STATS_EVERY == 0:
                print(f'Training loss: {loss.mean().item() * GRADIENT_ACCUMULATE_EVERY}')
                print(f'Training acc: {acc.mean().item()}')

            train_losses.append(loss.mean().item() * GRADIENT_ACCUMULATE_EVERY)
            train_accs.append(acc.mean().item())

            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optim.step()

            nsteps += 1

            if i % VALIDATE_EVERY == 0:
                model.eval()
                with torch.no_grad():
                        val_loss, val_acc = model(next(val_loader))

                        print(f'Validation loss: {val_loss.mean().item()}')
                        print(f'Validation acc: {val_acc.mean().item()}')

                        val_losses.append(val_loss.mean().item())
                        val_accs.append(val_acc.mean().item())

                        print('Plotting training loss graph...')

                        tr_loss_list = train_losses
                        plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
                        plt.show()
                        plt.close()
                        print('Done!')

                        print('Plotting training acc graph...')

                        tr_loss_list = train_accs
                        plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
                        plt.show()
                        plt.close()
                        print('Done!')

                        print('Plotting validation loss graph...')
                        tr_loss_list = val_losses
                        plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
                        plt.show()
                        plt.close()
                        print('Done!')

                        print('Plotting validation acc graph...')
                        tr_loss_list = val_accs
                        plt.plot([i for i in range(len(tr_loss_list))] ,tr_loss_list, 'b')
                        plt.show()
                        plt.close()
                        print('Done!')

                model.train()

            if i % GENERATE_EVERY == 0:
                model.eval()

                inp = random.choice(val_dataset)[:1022]

                print(inp)

                sample = model.module.generate(inp[None, ...], GENERATE_LENGTH)

                print(sample)

                model.train()

            if i % SAVE_EVERY == 0:

                print('Saving model progress. Please wait...')
                print('model_checkpoint_' + str(nsteps) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth')

                fname = '/home/ubuntu/model_checkpoint_' + str(nsteps) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth'

                torch.save(model.state_dict(), fname)

                data = [train_losses, train_accs, val_losses, val_accs]

                TMIDIX.Tegridy_Any_Pickle_File_Writer(data, '/home/ubuntu/losses_accs')

                print('Done!')

# Final Save

In [ ]:
print('Saving model progress. Please wait...')
print('model_checkpoint_' + str(nsteps) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth')

fname = '/home/ubuntu/model_checkpoint_' + str(nsteps) + '_steps_' + str(round(float(train_losses[-1]), 4)) + '_loss_' + str(round(float(train_accs[-1]), 4)) + '_acc.pth'

torch.save(model.state_dict(), fname)

print('Done!')

In [ ]:
data = [train_losses, train_accs, val_losses, val_accs]

TMIDIX.Tegridy_Any_Pickle_File_Writer(data, '/home/ubuntu/losses_accuracies')

In [ ]:
# Save training loss graph

plt.plot([i for i in range(len(train_losses))] ,train_losses, 'b')
plt.savefig('/home/ubuntu/training_loss_graph.png')
plt.close()
print('Done!')

# Save training acc graph

plt.plot([i for i in range(len(train_accs))] ,train_accs, 'b')
plt.savefig('/home/ubuntu/training_acc_graph.png')
plt.close()
print('Done!')

# Save validation loss graph

plt.plot([i for i in range(len(val_losses))] ,val_losses, 'b')
plt.savefig('/home/ubuntu/validation_loss_graph.png')
plt.close()
print('Done!')

# Save validation acc graph

plt.plot([i for i in range(len(val_accs))] ,val_accs, 'b')
plt.savefig('/home/ubuntu/validation_acc_graph.png')
plt.close()
print('Done!')

# Eval

In [ ]:
model.eval()

#x = (torch.tensor(random.choice(train_data)[:1022], dtype=torch.long, device=device_type)[None, ...])
x = torch.tensor([[937]] * 4, dtype=torch.long, device='cuda')

# run generation

out = model.module.generate(x,
                            1021,
                            temperature=0.9,
                            return_prime=True,
                            verbose=True)

y = out.tolist()

print('---------------')

In [ ]:
print(y)

In [ ]:
#@title Test INTs

data = y[0]

print('Sample INTs', data[:15])

out = data[:200000]

if len(out) != 0:

    song = out
    song_f = []

    time = 0
    dur = 0
    vel = 90
    pitch = 0
    channel = 0

    for ss in song:

        if 0 <= ss < 128:

            time += ss

        if 128 < ss < 256:

            dur = (ss-128)

        if 256 < ss < 512:

            chan = (ss-256) // 128

            if chan == 1:
                channel = 9
            else:
                channel = 0

            pitch = (ss-256) % 128

            song_f.append(['note', time, dur, channel, pitch, vel ])

detailed_stats = TMIDIX.Tegridy_ms_SONG_to_MIDI_Converter(song_f,
                                                          output_signature = 'Ultimate MIDI Classifier',
                                                          output_file_name = '/home/ubuntu/Ultimate-MIDI-Classifier-Composition',
                                                          track_name='Project Los Angeles',
                                                          timings_multiplier=32
                                                          )

print('Done!')

In [ ]:
tok_emb = model.module.net.token_emb.emb.weight.detach().cpu().tolist()

cos_sim = metrics.pairwise_distances(
  tok_emb, metric='cosine'
)
plt.figure(figsize=(7, 7))
plt.imshow(cos_sim, cmap="inferno", interpolation="nearest")
im_ratio = cos_sim.shape[0] / cos_sim.shape[1]
plt.colorbar(fraction=0.046 * im_ratio, pad=0.04)
plt.xlabel("Position")
plt.ylabel("Position")
plt.tight_layout()
plt.plot()
plt.savefig("/home/ubuntu/Ultimate-MIDI-Classifier-Tokens-Embeddings-Plot.png", bbox_inches="tight")

# Congrats! You did it! :)